In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [ ]:
# URL de la página a hacer scraping
url = 'https://www.meff.es/esp/Derivados-Financieros/Ficha/FIEM_MiniIbex_35'

# Realizar la petición HTTP GET a la página
response = requests.get(url)

In [10]:
def obtener_dataframe(response, tipo_tabla):
    """
    Realiza el web scraping y devuelve un dataframe con los datos obtenidos,
    dependiendo si el tipo de tabla es 'opciones' o 'futuros'.
    
    Args:
    - response: La respuesta HTTP obtenida.
    - tipo_tabla: Tipo de la tabla a buscar ('opciones' o 'futuros').
    
    Returns:
    - Un dataframe con los datos de la tabla.
    """
    # Determinar el ID de la tabla y si se necesita manejar el atributo data-tipo
    if tipo_tabla == 'opciones':
        id_tabla = 'tblOpciones'
        es_opcion = True
    elif tipo_tabla == 'futuros':
        id_tabla = 'Contenido_Contenido_tblFuturos'
        es_opcion = False
    else:
        print("Tipo de tabla no soportado.")
        return pd.DataFrame()
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', id=id_tabla)
        all_rows_data = []
        
        if table:
            rows = table.find_all('tr', class_='text-right')
            
            for row in rows:
                cells = row.find_all('td')
                row_data = [cell.text.strip() for cell in cells]
                
                if es_opcion:
                    data_tipo = row.get('data-tipo', 'No especificado')
                    row_data.insert(0, data_tipo)
                    
                all_rows_data.append(row_data)
                
            return pd.DataFrame(all_rows_data)
        else:
            print('No se encontró la tabla con el id especificado.')
            return pd.DataFrame()
    else:
        print('Error al realizar la petición HTTP:', response.status_code)
        return pd.DataFrame()

In [11]:
def tratar_dataframe(df, tipo_tabla):
    """
    Transforma el dataframe según si es de opciones o de futuros.
    
    Args:
    - df: Dataframe a transformar.
    - tipo_tabla: Tipo de la tabla ('opciones' o 'futuros').
    
    Returns:
    - Un dataframe transformado.
    """
    if tipo_tabla == 'opciones':
        # Especificar los nombres de columna para opciones
        df.columns = ['Class', 'Strike', 'Buy_ord', 'Buy_vol', 'Buy_price', 'Sell_price', 'Sell_vol', 'Sell_ord', 'Ult', 'Vol', 'Aper', 'Max.', 'Min.','Ant']
        df['Tipo'] = df['Class'].str[:3]
        df['Fecha'] = pd.to_datetime(df['Class'].str[3:], format='%Y%m%d').dt.strftime('%d-%m-%Y')
        df = df.drop(['Class'], axis=1)
        column_order = ['Tipo', 'Fecha'] + [col for col in df.columns if col not in ['Tipo', 'Fecha']]
    elif tipo_tabla == 'futuros':
        # Especificar los nombres de columna para futuros
        df.columns = ['Vencimiento', 'Tipo', 'Buy_ord', 'Buy_vol', 'Buy_price', 'Sell_price', 'Sell_vol', 'Sell_ord', 'Ult', 'Vol', 'Aper', 'Max.', 'Min.','Ant']
        column_order = ['Tipo', 'Vencimiento'] + [col for col in df.columns if col not in ['Tipo', 'Vencimiento']]
    else:
        print("Tipo de tabla no soportado.")
        return pd.DataFrame()
    
    df = df[column_order]
    
    return df

In [15]:
# Opciones
df_opciones = obtener_dataframe(response, 'opciones')
df_opciones = tratar_dataframe(df_opciones, 'opciones')

In [17]:
# Futuros
df_futuros = obtener_dataframe(response, 'futuros')
df_futuros = tratar_dataframe(df_futuros, 'futuros')

In [18]:
df_futuros

,Tipo,Vencimiento,Buy_ord,Buy_vol,Buy_price,Sell_price,Sell_vol,Sell_ord,Ult,Vol,Aper,Max.,Min.,Ant
0,Difer.,19 abr. 2024,10,25,"10.855,00","10.860,00",36,11,"10.860,00",411,"10.860,00","10.885,00","10.840,00","10.827,10"
1,Difer.,17 may. 2024,1,15,"10.815,00","10.830,00",15,1,"10.810,00",11,"10.835,00","10.840,00","10.810,00","10.792,00"
2,Difer.,21 jun. 2024,1,1,"10.815,00","10.845,00",1,1,-,0,-,-,-,"10.802,00"
